In [1]:
import qiskit
import numpy as np
from shadow_clifford import ClassicalShadow
from qiskit.quantum_info import Pauli, SparsePauliOp
from scipy.linalg import eigh
import pandas as pd

def eigensolver(s, hoff, hdiag):
    S = np.array([[1, s.real],[s.real,1]])
    H = np.array([[hdiag.real, hoff.real],[hoff.real, hdiag.real]])
    eigvals = eigh(H, S, eigvals_only=True, subset_by_index=[0, 1])
    return eigvals

def dep(p, rho):
    rho = np.array(rho)
    n = int(np.log2(np.shape(rho)[0]))
    return qiskit.quantum_info.DensityMatrix((1-p)*rho + p*np.eye(2**n)/2**n)

ref1 = [-4.04428374e-17+6.84026750e-17j,  1.57689901e-16-2.09549877e-16j,
       -2.54203437e-16+2.37034147e-16j,  9.89665456e-01+1.11022302e-15j,
        8.53310932e-17+4.89498545e-17j, -5.10409563e-17-7.50020036e-18j,
        1.05562794e-15-4.66843267e-16j, -1.09781758e-17-2.94116390e-17j,
       -1.24394555e-17-4.33982534e-17j, -4.90248239e-16-1.92205010e-17j,
       -6.78313947e-17-1.24724479e-16j,  9.25211277e-17-9.85027359e-17j,
       -1.43395555e-01-2.21031896e-16j,  1.38634603e-16-9.15119396e-17j,
       -1.00178491e-16-7.90560994e-17j,  2.81287772e-17-7.04662818e-17j]
rho1 = qiskit.quantum_info.DensityMatrix(ref1)

ref2 = [ 2.78547601e-18-2.49498567e-17j, -1.58742474e-16-3.55330186e-16j,
        2.16609908e-16-2.82403633e-16j, -6.10622664e-16-8.65265655e-01j,
        6.31302052e-17+1.60032916e-16j,  7.35833269e-17+1.93167449e-16j,
       -2.77555756e-16-2.99667310e-01j, -4.14374502e-16+1.62537109e-19j,
        7.98240847e-17-6.73495382e-17j, -1.11022302e-16-2.99667310e-01j,
       -1.71427315e-17+9.14602200e-17j,  1.10302771e-16-4.22197611e-16j,
       -1.11022302e-16+2.67795357e-01j,  1.21205829e-16+9.25910513e-17j,
       -2.96046751e-17-1.50472834e-16j, -1.33450810e-16+5.78675791e-18j]
rho2 = qiskit.quantum_info.DensityMatrix(ref2)

# Define the terms in the Hamiltonian
terms = [
    (-0.4196023680844313, 'IIII'),
    (-0.0444822913474488, 'XXYY'),
    (0.0444822913474488, 'XYYX'),
    (0.03837663252892021, 'XZXI'),
    (-0.01989799759163713, 'XZXZ'),
    (0.018478634752613642, 'XIXI'),
    (0.0444822913474488, 'YXXY'),
    (-0.0444822913474488, 'YYXX'),
    (0.03837663252892021, 'YZYI'),
    (-0.01989799759163713, 'YZYZ'),
    (0.018478634752613642, 'YIYI'),
    (0.10933758934583135, 'ZIII'),
    (-0.018478634752613635, 'ZXZX'),
    (-0.018478634752613635, 'ZYZY'),
    (0.14058793568268865, 'ZZII'),
    (0.09604367392140638, 'ZIZI'),
    (0.15645711712241647, 'ZIIZ'),
    (-0.03837663252892032, 'IXZX'),
    (0.019897997591637115, 'IXIX'),
    (-0.03837663252892032, 'IYZY'),
    (0.019897997591637115, 'IYIY'),
    (0.10933758934583139, 'IZII'),
    (0.15645711712241644, 'IZZI'),
    (0.09604367392140638, 'IZIZ'),
    (-0.07555373828970036, 'IIZI'),
    (0.14742615730183484, 'IIZZ'),
    (-0.07555373828970031, 'IIIZ')
]

simplified_terms = [(-0.6365898561329018, "IIII"), (-0.17792916538979606, "XXYY"), (0.153506530287386, "XZXI"), (0.1535065227735808, "XIXI"),(0.36978266455828424, "ZIII")]

reversed_terms = [(coeff, label[::-1]) for coeff, label in terms]
hamiltonian = sum(coeff * SparsePauliOp(Pauli(label)) for coeff, label in reversed_terms)
hamiltonian_matrix = hamiltonian.to_matrix()

simplified_hamiltonian = sum(coeff * SparsePauliOp(Pauli(label[::-1])) for coeff, label in simplified_terms)
simplified_hamiltonian_matrix = simplified_hamiltonian.to_matrix()

s_off_true = np.sqrt(np.trace(rho1.to_operator() @ rho2.to_operator()))
h_diag_true = np.trace(rho1.to_operator() @ hamiltonian_matrix)
h_s_true = np.trace(rho1.to_operator() @ rho2.to_operator() @ hamiltonian_matrix)
h_off_true = h_s_true/s_off_true
s0_true, t1_true = eigensolver(s_off_true, h_off_true, h_diag_true)

def print_metrics(s_off, h_off, h_diag, s0_list, t1_list, em = False):
    mean_s_off = np.mean(s_off)
    std_s_off = np.std(s_off)
    error_s_off = np.mean([abs(x - s_off_true) for x in s_off])

    mean_h_off = np.mean(h_off)
    std_h_off = np.std(h_off)
    error_h_off = np.mean([abs(x - h_off_true) for x in h_off])

    mean_h_diag = np.mean(h_diag)
    std_h_diag = np.std(h_diag)
    error_h_diag = np.mean([abs(x - h_diag_true) for x in h_diag])

    mean_s0 = np.mean(s0_list)
    std_s0 = np.std(s0_list)
    error_s0 = np.mean([abs(x - s0_true) for x in s0_list])

    mean_t1 = np.mean(t1_list)
    std_t1 = np.std(t1_list)
    error_t1 = np.mean([abs(x - t1_true) for x in t1_list])

    if em == None:
        print("Metrics     | S_off    | H_off    | H_diag  | S0      | T1")
        print("------------|----------|----------|---------|---------|----")
        print("True        | 0.895    | -0.954   | -1.042  | -1.0535 | -0.828")
        print(f"Mean        | {mean_s_off:.4f}   | {mean_h_off.real:.4f}  | {mean_h_diag.real:.4f} | {mean_s0:.4f} | {mean_t1:.4f}")
        print(f"STD         | {std_s_off:.4f}   | {std_h_off:.4f}   | {std_h_diag.real:.4f}  | {std_s0:.4f}  | {std_t1:.4f}")
        print(f"Error       | {error_s_off:.4f}   | {error_h_off:.4f}   | {error_h_diag:.4f}  | {error_s0:.4f}  | {error_t1:.4f}")
    else:
        em_s_off, em_h_off, em_h_diag, em_s0, em_t1 = em
        em_mean_s_off = np.mean(em_s_off)
        em_std_s_off = np.std(em_s_off)
        em_error_s_off = np.mean([abs(x - s_off_true) for x in em_s_off])
        em_mean_h_off = np.mean(em_h_off)
        em_std_h_off = np.std(em_h_off)
        em_error_h_off = np.mean([abs(x - h_off_true) for x in em_h_off])
        em_mean_h_diag = np.mean(em_h_diag)
        em_std_h_diag = np.std(em_h_diag)
        em_error_h_diag = np.mean([abs(x - h_diag_true) for x in em_h_diag])
        em_mean_s0 = np.mean(em_s0)
        em_std_s0 = np.std(em_s0)
        em_error_s0 = np.mean([abs(x - s0_true) for x in em_s0])
        em_mean_t1 = np.mean(em_t1)
        em_std_t1 = np.std(em_t1)
        em_error_t1 = np.mean([abs(x - t1_true) for x in em_t1])

        print("Metrics     | S_off    | H_off    | H_diag  | S0      | T1")
        print("------------|----------|----------|---------|---------|----")
        print("True        | 0.895    | -0.954   | -1.042  | -1.0535 | -0.828")
        print(f"Mean        | {mean_s_off:.4f}   | {mean_h_off.real:.4f}  | {mean_h_diag.real:.4f} | {mean_s0:.4f} | {mean_t1:.4f}")
        print(f"STD         | {std_s_off:.4f}   | {std_h_off:.4f}   | {std_h_diag.real:.4f}  | {std_s0:.4f}  | {std_t1:.4f}")
        print(f"Error       | {error_s_off:.4f}   | {error_h_off:.4f}   | {error_h_diag:.4f}  | {error_s0:.4f}  | {error_t1:.4f}")
        print("------------|----------|----------|---------|---------|----")
        print("EM          | S_off    | H_off    | H_diag  | S0      | T1")
        print("------------|----------|----------|---------|---------|----")
        print(f"Mean        | {em_mean_s_off.real:.4f}   | {em_mean_h_off.real:.4f}  | {em_mean_h_diag.real:.4f} | {em_mean_s0:.4f} | {em_mean_t1:.4f}")
        print(f"STD         | {em_std_s_off:.4f}   | {em_std_h_off:.4f}   | {em_std_h_diag.real:.4f}  | {em_std_s0:.4f}  | {em_std_t1:.4f}")
        print(f"Error       | {em_error_s_off:.4f}   | {em_error_h_off:.4f}   | {em_error_h_diag:.4f}  | {em_error_s0:.4f}  | {em_error_t1:.4f}")

print(f"s0_shadow = {s0_true}")
print(f"t1_shadow = {t1_true}")
print(f"s_off_shadow = {s_off_true}")
print(f"h_off_shadow = {h_off_true}")
print(f"h_diag_shadow = {h_diag_true}")

s0_shadow = -1.0535137399660863
t1_shadow = -0.8284433437356912
s_off_shadow = (0.8947241928601519-3.7884710497051973e-32j)
h_off_shadow = (-0.9544514644714568-6.7504896955333126e-18j)
h_diag_shadow = (-1.0416665061528663+2.7733391199176196e-32j)


In [2]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.compiler import transpile
import qiskit.providers.aer.noise as noise
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel

noise_model = NoiseModel()

prob_1 = 0.00003  # 1-qubit gate
prob_2 = 0.0015 # 2-qubit gate

error_1_dep = noise.depolarizing_error(prob_1, 1)
error_2_dep = noise.depolarizing_error(prob_2, 2)

error_1_amp  = noise.amplitude_damping_error(prob_1, 1)
error_2_amp_tmp = noise.amplitude_damping_error(prob_2, 1)
error_2_amp = error_2_amp_tmp.expand(error_2_amp_tmp)

error_1_phase = noise.phase_damping_error(prob_1, 1)
error_2_phase_tmp = noise.phase_damping_error(prob_2, 1)
error_2_phase = error_2_phase_tmp.expand(error_2_phase_tmp)

noise_model.add_all_qubit_quantum_error(error_1_dep, ['u3'])
noise_model.add_all_qubit_quantum_error(error_2_dep, ['cx'])
noise_model.add_all_qubit_quantum_error(error_1_amp, ['u3'])
noise_model.add_all_qubit_quantum_error(error_2_amp, ['cx'])
noise_model.add_all_qubit_quantum_error(error_1_phase, ['u3'])
noise_model.add_all_qubit_quantum_error(error_2_phase, ['cx'])

ansatz1 =  QuantumCircuit().from_qasm_str(open('ansatzes0_simplified.qasm', 'r').read())
ref1_circ = QuantumCircuit(4,4)
ref1_circ.x(0)
ref1_circ.x(1)
ref1_circ = ref1_circ.compose(ansatz1)
ref1_circ = transpile(ref1_circ, basis_gates=['u3', 'cx'], optimization_level=3)
ref1_circ.save_density_matrix()
ref1_result= execute(ref1_circ, backend =  AerSimulator(noise_model = noise_model, method = "density_matrix")).result()
dm1 = ref1_result.data()['density_matrix']

ansatz2 =  QuantumCircuit().from_qasm_str(open('ansatzes1_simplified.qasm', 'r').read())
ref2_circ = QuantumCircuit(4,4)
ref2_circ.x(0)
ref2_circ.x(1)
ref2_circ = ref2_circ.compose(ansatz2)
ref2_circ = transpile(ref2_circ, basis_gates=['u3', 'cx'], optimization_level=3)
ref2_circ.save_density_matrix()
ref2_result= execute(ref2_circ, backend =  AerSimulator(noise_model = noise_model, method = "density_matrix")).result()
dm2 = ref2_result.data()['density_matrix']
ref1_circ.count_ops()

OrderedDict([('u3', 113), ('cx', 63), ('save_density_matrix', 1)])

In [3]:
total_shots = [1e2, 1e3, 1e4, 1e5, 1e6, 1e7]
param_list = [(int(np.sqrt(x)), int(np.sqrt(x))) for x in total_shots]

rho1 = qiskit.quantum_info.DensityMatrix(dm1)
rho2 = qiskit.quantum_info.DensityMatrix(dm2)

s_off_list = []
h_off_list = []
h_diag_list = []
s0_list = []
t1_list = []
em_s_off_list = []
em_h_off_list = []
em_h_diag_list = []
em_s0_list = []
em_t1_list = []

for (num_repeats, num_shots) in param_list:
    s_off = []
    h_off = []
    h_diag = []
    s0 = []
    t1 = []
    em_s_off = []
    em_h_off = []
    em_h_diag = []
    em_s0 = []
    em_t1 = []

    # 11 hours
    i = 0
    while i < 10:

        cs1 = ClassicalShadow(rho1)
        cs1.collect(repeats=num_repeats, shots=num_shots)
        dm1 = np.array(cs1.recover_density())

        cs2 = ClassicalShadow(rho2)
        cs2.collect(repeats=num_repeats, shots=num_shots)
        dm2 = np.array(cs2.recover_density())

        s_off_tmp = np.sqrt(np.trace(dm1 @ dm2).real)
        h_diag_tmp = cs1.estimate(hamiltonian_matrix).real
        h_off_tmp = np.trace(dm1 @ dm2 @ hamiltonian_matrix)/s_off_tmp

        # em_factor = np.trace(dm1 @ dm1) * np.trace(dm2 @ dm2)
        # em_s_off_tmp = np.sqrt(np.trace(dm1 @ dm1 @ dm2 @ dm2)/em_factor)
        # em_h_diag_tmp = np.trace(dm1 @ dm1 @ hamiltonian_matrix)/np.trace(dm1 @ dm1)
        # em_h_off_tmp = np.trace(dm1 @ dm1 @ dm2 @ dm2 @ hamiltonian_matrix)/em_factor/em_s_off_tmp

        # distilled to the 3rd order!!!!!!!!!!!!!!!!!
        em_factor = np.trace(dm1 @ dm1 @ dm1) * np.trace(dm2 @ dm2 @ dm2)
        em_s_off_tmp = np.sqrt(np.trace(dm1 @ dm1 @ dm1 @ dm2 @ dm2 @ dm2).real/em_factor)
        em_h_diag_tmp = np.trace(dm1 @ dm1 @ dm1 @ hamiltonian_matrix)/np.trace(dm1 @ dm1 @ dm1)
        em_h_off_tmp = np.trace(dm1 @ dm1 @ dm1 @ dm2 @ dm2 @ dm2 @ hamiltonian_matrix)/em_factor/em_s_off_tmp

        try:
            eigvals = eigensolver(s_off_tmp, h_off_tmp, h_diag_tmp)
            s_off.append(s_off_tmp.real)
            h_diag.append(h_diag_tmp.real)
            h_off.append(h_off_tmp.real)
            s0.append(eigvals[0])
            t1.append(eigvals[1])

            em_eigvals = eigensolver(em_s_off_tmp, em_h_off_tmp, em_h_diag_tmp)
            em_s0.append(em_eigvals[0])
            em_t1.append(em_eigvals[1])
            em_s_off.append(em_s_off_tmp.real)
            em_h_off.append(em_h_off_tmp.real)
            em_h_diag.append(em_h_diag_tmp.real)

            print(i)
            i += 1
        except:
            print("failed")
    print("done")
    s_off_list.append(s_off)
    h_off_list.append(h_off)
    h_diag_list.append(h_diag)
    s0_list.append(s0)
    t1_list.append(t1)
    em_s_off_list.append(em_s_off)
    em_h_off_list.append(em_h_off)
    em_h_diag_list.append(em_h_diag)
    em_s0_list.append(em_s0)
    em_t1_list.append(em_t1)

0
1


C:\Users\17102\AppData\Local\Temp\ipykernel_9028\3395064523.py:42: RuntimeWarning: invalid value encountered in sqrt
  s_off_tmp = np.sqrt(np.trace(dm1 @ dm2).real)
C:\Users\17102\AppData\Local\Temp\ipykernel_9028\3395064523.py:48: RuntimeWarning: invalid value encountered in scalar divide
  h_off_tmp = np.trace(dm1 @ dm2 @ hamiltonian_matrix)/s_off_tmp


failed
2
failed
3
4
failed
5
6
7
8
9
done
0
failed
1
2
3
4
5
6
7
8
9
done
0
1
2
3
4
5
6
7
8
9
done
0
1
2
3
4
5
6
7
8
9
done
0
1
2
3
4
5
6
7
8
9
done
0
1
2
3
4
5
6
7
8
9
done


In [13]:
# save data
data = {
    "s_off": s_off_list,
    "h_off": h_off_list,
    "h_diag": h_diag_list,
    "s0": s0_list,
    "t1": t1_list,
    "em_s_off": em_s_off_list,
    "em_h_off": em_h_off_list,
    "em_h_diag": em_h_diag_list,
    "em_s0": em_s0_list,
    "em_t1": em_t1_list
}
df = pd.DataFrame(data)
df.to_csv("data/shadow_noisy.csv")

In [9]:
mean_s_off = [np.mean(s) for s in s_off_list]
std_s_off = [np.std(s) for s in s_off_list]
mean_h_off = [np.mean(h) for h in h_off_list]
std_h_off = [np.std(h) for h in h_off_list]
mean_h_diag = [np.mean(h) for h in h_diag_list]
std_h_diag = [np.std(h) for h in h_diag_list]
mean_s0 = [np.mean(s) for s in s0_list]
std_s0 = [np.std(s) for s in s0_list]
mean_t1 = [np.mean(t) for t in t1_list]
std_t1 = [np.std(t) for t in t1_list]

mean_s_off_em = [np.mean(s) for s in em_s_off_list]
std_s_off_em = [np.std(s) for s in em_s_off_list]
mean_h_off_em = [np.mean(h) for h in em_h_off_list]
std_h_off_em = [np.std(h) for h in em_h_off_list]
mean_h_diag_em = [np.mean(h) for h in em_h_diag_list]
std_h_diag_em = [np.std(h) for h in em_h_diag_list]
mean_s0_em = [np.mean(s) for s in em_s0_list]
std_s0_em = [np.std(s) for s in em_s0_list]
mean_t1_em = [np.mean(t) for t in em_t1_list]
std_t1_em = [np.std(t) for t in em_t1_list]

In [14]:
unmitigated_data = {"Mean S_off": mean_s_off, "STD S_off": std_s_off, "Mean H_off": mean_h_off, "STD H_off": std_h_off, "Mean H_diag": mean_h_diag, "STD H_diag": std_h_diag, "Mean S0": mean_s0, "STD S0": std_s0, "Mean T1": mean_t1, "STD T1": std_t1}
unmitigated_data_df = pd.DataFrame(unmitigated_data)
mitigated_data = {"Mean S_off": mean_s_off_em, "STD S_off": std_s_off_em, "Mean H_off": mean_h_off_em, "STD H_off": std_h_off_em, "Mean H_diag": mean_h_diag_em, "STD H_diag": std_h_diag_em, "Mean S0": mean_s0_em, "STD S0": std_s0_em, "Mean T1": mean_t1_em, "STD T1": std_t1_em}
mitigated_data_df = pd.DataFrame(mitigated_data)
print(unmitigated_data_df, "\n", mitigated_data_df)

   Mean S_off  STD S_off  Mean H_off  STD H_off  Mean H_diag  STD H_diag  \
0    0.613347   0.133912   -0.623733   0.182301    -0.874620    0.340681   
1    0.706492   0.101937   -0.758085   0.123925    -0.964169    0.146573   
2    0.753181   0.061866   -0.802783   0.064743    -0.958329    0.105118   
3    0.740049   0.038921   -0.788573   0.038006    -0.956121    0.083311   
4    0.734105   0.022200   -0.778682   0.023438    -0.943525    0.034190   
5    0.727671   0.009415   -0.773322   0.009872    -0.950048    0.020946   

    Mean S0    STD S0   Mean T1    STD T1  
0 -1.282303  1.184710 -0.458041  0.533906  
1 -1.083044  0.172872 -0.596618  0.485271  
2 -1.026570  0.091517 -0.640342  0.395706  
3 -1.036927  0.138596 -0.633169  0.255904  
4 -0.993104  0.020316 -0.617345  0.135048  
5 -0.997460  0.012284 -0.650513  0.068813   
    Mean S_off  STD S_off  Mean H_off  STD H_off  Mean H_diag  STD H_diag  \
0    0.271010   0.016076   -0.176365   0.040462    -0.497556    0.062244   
1    

In [4]:
# use pandas to store the data
#shadow_unmitigated = pd.DataFrame({'s_off': s_off, 'h_off': h_off, 'h_diag': h_diag, 's0': s0, 't1': t1})
#shadow_mitigated = pd.DataFrame({'em_s_off': em_s_off, 'em_h_off': em_h_off, 'em_h_diag': em_h_diag, 'em_s0': em_s0, 'em_t1': em_t1})

In [5]:
# save data to the data folder
#shadow_unmitigated.to_csv('data/shadow_unmitigated.csv')
#shadow_mitigated.to_csv('data/shadow_mitigated.csv')